# 1. LOADING

In [1]:
# import packages
import pandas as pd
import graphlab as gl
from sklearn import metrics
from scipy import stats

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 50)

In [3]:
# print variable names
#data.columns

# 2. DATA PREPARATION

In [4]:
# converting factors to strings
data["user_id"]   = data.user_id.astype(str)
data["media_id"]  = data.media_id.astype(str)
data["album_id"]  = data.album_id.astype(str)
data["artist_id"] = data.artist_id.astype(str)
data["genre_id"]  = data.genre_id.astype(str)

In [5]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 50)
test: (39821, 50)
known: (2319611, 50)
unknown: (19918, 50)


In [6]:
# data sets with user-media interactions
tr_listen = gl.SFrame(tr[["media_id", "user_id", "is_listened"]])
ts_listen = gl.SFrame(ts[["media_id", "user_id", "is_listened"]])
kn_listen = gl.SFrame(kn[["media_id", "user_id", "is_listened"]])
un_listen = gl.SFrame(un[["media_id", "user_id", "is_listened"]])

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1495905616.log


This non-commercial license of GraphLab Create for academic use is assigned to kozodoin@hu-berlin.de and will expire on May 12, 2018.


In [7]:
# clearing the memory
data = "Null"
tr   = "Null"
kn   = "Null"

# 3. MODELING

## 3.1. TRAINING-VALIDATION

In [8]:
# model parameters
k = 20000
sim_metric = "pearson"

In [9]:
# training the model
model = gl.recommender.item_similarity_recommender.create(tr_listen, 
                                                          only_top_k = k,
                                                          similarity_type = sim_metric, 
                                                          user_id = "user_id", 
                                                          item_id = "media_id", 
                                                          target  = "is_listened")

Recsys training: model = item_similarity

Preparing data set.

Data has 2279790 observations with 14403 users and 149282 items.

Data prepared in: 1.89235s

Training model from provided data.

...

| 1m 23s                              | 100              | 149282          |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 93.4589s

In [10]:
# forecasting
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict(ts_listen)
pred.head(3)

,row_index,is_listened
2166,6060544,0.456872
2167,3968455,0.638511
2239,5584470,0.646514


In [11]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred["is_listened"])

0.58372875796077661

In [12]:
# saving prediction vector
pred.to_csv(path + "pred_valid/similarity_scores_flow_pearson_20000k.csv", index = False)

In [ ]:
# clearing the memory
tr_listen = "Null"
ts = "Null"

## 3.2. KNOWN-UNKNOWN

In [ ]:
# training the model
model = gl.recommender.item_similarity_recommender.create(kn_listen, 
                                                          only_top_k = k,
                                                          similarity_type = sim_metric, 
                                                          user_id = "user_id", 
                                                          item_id = "media_id", 
                                                          target  = "is_listened")

Recsys training: model = item_similarity

Preparing data set.

Data has 2319611 observations with 15971 users and 151720 items.

Data prepared in: 2.73086s

Training model from provided data.

...

+--------------------------------+------------+

| 8.009ms                        | 6.25       |

| 72.143ms                       | 100        |

+--------------------------------+------------+

Setting up lookup tables.

In [ ]:
# forecasting
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict(un_listen)
pred = pred.sort_values("sample_id")
pred.head(3)

In [ ]:
# saving prediction vector
pred.to_csv(path + "pred_unknown/similarity_scores_flow_pearson_20000k.csv", index = False)